<a href="https://colab.research.google.com/github/sonali-singh97/GyneConnect/blob/main/Hack_The_Gap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl 
from openpyxl.utils import range_boundaries

Link to original data collected by Amba Azaad : https://docs.google.com/spreadsheets/d/1o2QJKrPjKS92cCj6SX73B8siKlW6TlRNKYe0SC7WrX8/edit#gid=1344865269

After doing some data cleaning, this is the spreadsheet -> ////add link///////////

Workbook has different cities on different sheets, loading each sheet as a dataframe and storing it in a dictionary 'data' with city name as a key:

In [45]:
data = pd.read_excel('/content/drive/MyDrive/Hack The Gap/Crowdsourced List of Gynaecologists We Can Trust In India.xlsx',sheet_name=None,header=None)

These are the cities present in the data

In [3]:
data.keys()

dict_keys(['Abohar', 'Ahmedabad', 'Allahabad', 'Bengaluru', 'Bhopal', 'Chandigarh', 'Chennai', 'Coimbatore', 'Colombo', 'Delhi', 'Ghaziabad', 'Gurgaon', 'Gwalior', 'Hoshiarpur', 'Hyderabad', 'Indore', 'Jaipur', 'Kakinada', 'Kochi', 'Kolkata', 'Kurnool', 'Lucknow', 'Ludhiana', 'Mangaluru', 'Mumbai', 'Mysuru', 'Nagpur', 'Navi Mumbai', 'Noida', 'Panjim', 'Patiala', 'Pune', 'Ranchi', 'Secunderabad', 'Solapur', 'Surat', 'Thane', 'Thiruvananthapuram', 'Thrissur', 'Udaipur', 'Udupi', 'Vishakapatnam', 'Other Places'])

Printing shape of each dataframe (In original Amba's sheet, shapes were different, with data stored in different formats in different sheets) 

In [4]:
info = pd.DataFrame(columns=['City','Shape'])
for i,key in enumerate(data):
  info.loc[i] = key,data[key].shape

In [47]:
info

,City,Shape
0,Abohar,"(60, 2)"
1,Ahmedabad,"(60, 6)"
2,Allahabad,"(60, 3)"
3,Bengaluru,"(60, 98)"
4,Bhopal,"(60, 2)"
5,Chandigarh,"(60, 2)"
6,Chennai,"(60, 31)"
7,Coimbatore,"(60, 5)"
8,Colombo,"(60, 10)"
9,Delhi,"(60, 88)"


Combining all dataframes into single dataframe 'df', with city as an additional column:
* For multiple entries for the same doctor, some fields had been merged -> in the dataframe cell content of merged cell was filled in 1st cell of dataframe, and rest got null values hence filled null values with previous column value before appending
* we will do this only for name of doctor, since other answers will be unique to reviewer
* taking transpose
* resetting index 
* correcting column headers



In [99]:
df = data['Other Places'].T
df.reset_index(inplace=True)
df.columns = df.iloc[0]
df = df[1:]

In [100]:
for key in data:
  data[key][data[key].index==0] = data[key][data[key].index==0].fillna(axis=1,method='ffill')
  tmp = data[key].T
  tmp.reset_index(inplace=True)
  tmp.columns = tmp.iloc[0]
  tmp = tmp[1:]
  tmp['City'] = key
  df = df.append(tmp,ignore_index=True)

In [101]:
df.drop(0,axis=1,inplace=True)

Now, let's take a look at our combined dataset

In [102]:
df.head()

,City,Name,Gender,Age-Range,Office Tel. No.,Office Address,Office Hours,Standard Consultation Fees (INR),What forms of payment do they accept?,Language(s) Spoken,Is their office wheelchair accessible?,Other disability-accessibility,"Do they keep their office hygienic, clean, private and welcoming?",About the Reccer,Reccer's Testimonial,"Will they keep your information private, even from your parents or partner?","Will they treat you as autonomous, without demanding permission/consent from parents or partner?",Will they advocate for your legal rights to sexual and medical autonomy if challenged by your family members/spouse/employers?,Will they positively and non-judgementally respect your choice to be accompanied by someone who is not a biological parent or husband?,Will they make poor and/or illiterate folks feel welcomed?,Will they be considerate of your financial burden and not prescribe unnecessary tests to pad their wallet?,"Will they make folks from an oppressed religion, race, ethnicity or caste feel welcomed?",Will they respect your dietary choices?,Will they work with your desire for alternative therapies and medicine systems?,"Will they make trans men, trans women and other folks who don't identify as cis female feel welcomed?",Will they discuss gender dysphoria related medical issues positively and non-judgementally?,"Will they discuss queer sexuality (lesbian, bisexual, ace and others) positively and non-judgementally?","Will they discuss kink, polyamoury, and other forms of sexual expression without embarrassment or censure?","Will they welcome survivors of incest, sexual and domestic abuse, and discuss healing from rape and sexual trauma sensitively and respectfully?",Will they support children and teenagers regardless of their sexual history and place their well-being ahead of their guardian’s morality?,Will they welcome folks with AIDS or other sexually transmittable diseases?,"Will they discuss the sexuality of folks with physical disabilities without judgement, and accommodate their needs while prescribing treatment?","Will they discuss the sexuality of folks with mental illnesses without judgement, and accommodate their needs while prescribing treatment?",Will they discuss sex work positively and non-judgementally?,Will they frankly and without squeamishness advocate for your sexual satisfaction and pleasure?,Will they discuss masturbation positively and non-judgementally?,Will they discuss menstrual hygiene choices including cups and tampons without moral judgements?,Will they respect the importance you do or don't place on the condition of your hymen?,Will they look for solutions to your pain and illness without advocating marriage/pregnancy?,"Will they respectfully and non-judgmentally ask about and accept your sexual activity and history, regardless of marital status?",Will they discuss choosing to stay celibate or child-free positively and non-judgementally?,Will they give you options for birth control without advocating abstinence?,Will they prescribe you the morning after pill without judgement or hesitation?,"Will they respect your choice to keep or abort a pregnancy, regardless of your marital status?",Are they equipped for surgical abortions and post-termination monitoring and care?,"Will they prescribe, monitor and support pill-based abortions?",Will they respect the importance you do or don't place on the condition of your fertility?,Will they discuss infertility treatment options positively and non-judgementally?,"Will they support single parenting, surrogacy and non-normative parenting choices?",Will they refuse - even under pressure - to illegally reveal a foetus’s sex?,Will they support your choice of non-traditional birthing techniques?,Will they welcome your desired companion(s) into their delivery rooms?,Will they seek and respect your decision on any surgical/invasive procedure during delivery?,"Are they up-do-date in the field, aware of terms like endometriosis, sex toys, gender dysphori

In [103]:
df.shape

(450, 61)

Let's ensure that there are no null values in the two compulsary fields, name and city

In [104]:
df.isnull().sum().head(2)

0
City    0
Name    0
dtype: int64

In [105]:
df.columns

Index(['City', 'Name', 'Gender', 'Age-Range', 'Office Tel. No.',
       'Office Address', 'Office Hours', 'Standard Consultation Fees (INR)',
       'What forms of payment do they accept?', 'Language(s) Spoken',
       'Is their office wheelchair accessible?',
       'Other disability-accessibility',
       'Do they keep their office hygienic, clean, private and welcoming?',
       'About the Reccer', 'Reccer's Testimonial',
       'Will they keep your information private, even from your parents or partner? ',
       'Will they treat you as autonomous, without demanding permission/consent from parents or partner? ',
       'Will they advocate for your legal rights to sexual and medical autonomy if challenged by your family members/spouse/employers?',
       'Will they positively and non-judgementally respect your choice to be accompanied by someone who is not a biological parent or husband?',
       'Will they make poor and/or illiterate folks feel welcomed?',
       'Will they be cons

Now, creating a new dataframe for GyneConnect

In [106]:
gyne = pd.DataFrame(data = {'Name':df.Name,'City':df.City,'Gender':df.Gender,'Contact No':df['Office Tel. No.'],'Address':df['Office Address'],'Office Hours':df['Office Hours'],'Fees':df['Standard Consultation Fees (INR)'],'About Reccer':df['About the Reccer'],'Testimonial':df["Reccer's Testimonial"]},columns=['Name','City','Gender','Contact No','Address','Office Hours','Fees','TAG_auto','TAG_trans','TAG_queer','TAG_abuse','TAG_std','TAG_body','TAG_consent','About Reccer','Testimonial'])

In [107]:
gyne.TAG_queer = df['Will they discuss queer sexuality (lesbian, bisexual, ace and others) positively and non-judgementally?'].str.contains('Yes|Yep')
gyne.TAG_abuse = df['Will they welcome survivors of incest, sexual and domestic abuse, and discuss healing from rape and sexual trauma sensitively and respectfully?'].str.contains('Yes|Yep')
gyne.TAG_std = df['Will they welcome folks with AIDS or other sexually transmittable diseases?'].str.contains('Yes|Yep')
gyne.TAG_body = df['Will they examine your body thoroughly and professionally without making you feel ugly, embarrassed, ashamed or uncomfortable?'].str.contains('Yes|Yep')

In [108]:
gyne.head()

,Name,City,Gender,Contact No,Address,Office Hours,Fees,TAG_auto,TAG_trans,TAG_queer,TAG_abuse,TAG_std,TAG_body,TAG_consent,About Reccer,Testimonial
0,"Elizabeth Kass, MD",Boston,Female,(617) 267-0900,"Fenway Health 1340 Boylston St, Boston, MA 02215",NaN,NaN,NaN,NaN,True,True,True,True,NaN,NaN,NaN
1,Holly Ford,"Columbia, MO",Female,(573) 499-6084,"404 N Keene St Ste 3\nColumbia, MO 65201","8-5, M-F",NaN,NaN,NaN,True,True,True,True,NaN,NaN,NaN
2,Morvan Eluf,"Sao Paulo, Brasil",Male,NaN,NaN,NaN,250 reais (80$),NaN,NaN,True,True,True,True,NaN,NaN,NaN
3,Dr. Katherin,Zurich Switzerland,Female,+41 44 255 50 36,UniversitätsSpital Zürich\nFrauenpoliklinik B ...,NaN,Private Insurance,NaN,NaN,False,False,True,True,NaN,I am PhD Student at University of Zurich. I am...,I just want to say that gynecologists from Ind...
4,Lauren Pellegrino,New York City,Female,NaN,"The Door \n555 Broome Street, New York, NY",NaN,NaN,NaN,NaN,True,True,True,True,NaN,"I haven't found one I'm 100% happy with, but I...",NaN


Now, for the TAGs which require more than one question

In [109]:
gyne.TAG_trans = df["Will they make trans men, trans women and other folks who don't identify as cis female feel welcomed?"].str.contains('Yes|Yep') | df['Will they discuss gender dysphoria related medical issues positively and non-judgementally?'].str.contains('Yes|Yep') 

In [110]:
gyne.TAG_consent = df['Will they treat you as autonomous, without demanding permission/consent from parents or partner? '].str.contains('Yes|Yep') & df['Will they keep your information private, even from your parents or partner? '].str.contains('Yes|Yep')

In [111]:
gyne.TAG_auto = df['Will they look for solutions to your pain and illness without advocating marriage/pregnancy?'].str.contains('Yes|Yep') & df['Will they respectfully and non-judgmentally ask about and accept your sexual activity and history, regardless of marital status?'].str.contains('Yes|Yep') & df['Will they discuss choosing to stay celibate or child-free positively and non-judgementally?'].str.contains('Yes|Yep') & df['Will they treat you as autonomous, without demanding permission/consent from parents or partner? '].str.contains('Yes|Yep') 

In [112]:
gyne.head()

,Name,City,Gender,Contact No,Address,Office Hours,Fees,TAG_auto,TAG_trans,TAG_queer,TAG_abuse,TAG_std,TAG_body,TAG_consent,About Reccer,Testimonial
0,"Elizabeth Kass, MD",Boston,Female,(617) 267-0900,"Fenway Health 1340 Boylston St, Boston, MA 02215",NaN,NaN,True,True,True,True,True,True,True,NaN,NaN
1,Holly Ford,"Columbia, MO",Female,(573) 499-6084,"404 N Keene St Ste 3\nColumbia, MO 65201","8-5, M-F",NaN,True,True,True,True,True,True,True,NaN,NaN
2,Morvan Eluf,"Sao Paulo, Brasil",Male,NaN,NaN,NaN,250 reais (80$),True,False,True,True,True,True,True,NaN,NaN
3,Dr. Katherin,Zurich Switzerland,Female,+41 44 255 50 36,UniversitätsSpital Zürich\nFrauenpoliklinik B ...,NaN,Private Insurance,False,True,False,False,True,True,False,I am PhD Student at University of Zurich. I am...,I just want to say that gynecologists from Ind...
4,Lauren Pellegrino,New York City,Female,NaN,"The Door \n555 Broome Street, New York, NY",NaN,NaN,True,True,True,True,True,True,True,"I haven't found one I'm 100% happy with, but I...",NaN


In [114]:
gyne.to_json('doctors_data.json')

In [115]:
gyne.to_csv('doctors_data.csv',header=True)